# Modelado - Transformador

In [1]:
# Importaciones
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import resample
from sklearn.feature_selection import mutual_info_classif
from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.under_sampling import TomekLinks
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import time
from imblearn.base import SamplerMixin
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_recall_curve, auc

## 1. Utils

In [2]:
SEED = 9603

K_ITERS = 10

TRAIN_SIZE = 0.75

caracteristicas_numericas = [
  'flow_duration', 'total_fwd_packet', 'total_bwd_packets', 
  'total_length_of_fwd_packet', 'total_length_of_bwd_packet', 
  'fwd_packet_length_max', 'fwd_packet_length_min', 'fwd_packet_length_mean', 
  'fwd_packet_length_std', 'bwd_packet_length_max', 'bwd_packet_length_min', 
  'bwd_packet_length_mean', 'bwd_packet_length_std', 'flow_bytes/s', 
  'flow_packets/s', 'flow_iat_mean', 'flow_iat_std', 'flow_iat_max', 
  'flow_iat_min', 'fwd_iat_total', 'fwd_iat_mean', 'fwd_iat_std', 
  'fwd_iat_max', 'fwd_iat_min', 'bwd_iat_total', 'bwd_iat_mean', 
  'bwd_iat_std', 'bwd_iat_max', 'bwd_iat_min', 'fwd_psh_flags', 
  'bwd_psh_flags', 'fwd_header_length', 'bwd_header_length', 'fwd_packets/s', 
  'bwd_packets/s', 'packet_length_min', 'packet_length_max', 'packet_length_mean', 
  'packet_length_std', 'packet_length_variance', 'fin_flag_count', 'syn_flag_count',
  'rst_flag_count', 'psh_flag_count', 'ack_flag_count', 'cwr_flag_count', 
  'ece_flag_count', 'down/up_ratio', 'average_packet_size', 'fwd_segment_size_avg', 
  'bwd_segment_size_avg', 'fwd_bytes/bulk_avg', 'fwd_packet/bulk_avg', 
  'fwd_bulk_rate_avg', 'bwd_bytes/bulk_avg', 'bwd_packet/bulk_avg', 
  'bwd_bulk_rate_avg', 'subflow_fwd_packets', 'subflow_fwd_bytes', 
  'subflow_bwd_packets', 'subflow_bwd_bytes', 'fwd_init_win_bytes', 
  'bwd_init_win_bytes', 'fwd_act_data_pkts', 'fwd_seg_size_min', 
  'active_mean', 'active_std', 'active_max', 'active_min', 
  'idle_mean', 'idle_std', 'idle_max', 'idle_min'
]

caracteristicas_var_null = ['flow_bytes/s', 'flow_iat_mean', 'flow_iat_std', 'flow_iat_max', 'flow_iat_min']

caracteristicas_var_max_inf = ['flow_bytes/s', 'flow_packets/s']

caracteristicas_nominales = ['port_type_registered', 'port_type_well_known', 'protocol_6', 'protocol_17']

caracteristica_objetivo = "label"

## 2. Creación de transformador

In [3]:
class Transformer(BaseEstimator, TransformerMixin) : 
  def __init__(self, seed, categorical_chars, continous_chars, obj_char, obj_char_majority_class, selected_chars=None, imputer_strategy='median', threshold_redundance=0.9, inf_chars=[], null_chars=[], ordered_labels=[]):
    allowed_meassures = {'mean', 'median'}
    super().__init__()
    self.seed = seed 
    self.categorical_chars = categorical_chars 
    self.continous_chars = continous_chars
    self.obj_char = obj_char
    self.obj_char_majority_class = obj_char_majority_class
    self.selected_chars = selected_chars
    self.threshold_redundance = threshold_redundance
    self.inf_chars = inf_chars
    self.null_chars = null_chars
    self.ordered_labels = ordered_labels
    if imputer_strategy not in allowed_meassures : 
      raise ValueError(f"Valor inválido: {imputer_strategy}. Debe ser uno de {allowed_meassures}")
    self.imputer_strategy = imputer_strategy
    
  def inf_data_imputation(self, X_y : pd.DataFrame) : 
    X_y_ = X_y.copy()
    X_y_.replace([np.inf], np.nan, inplace=True)
    return X_y_

  def null_train_data_imputation(self, X_y : pd.DataFrame) :
    X_y_ = X_y.copy()
    for char in list(set(self.inf_chars+self.null_chars)) : 
      mask = X_y_[char].isna()
      # Se obtienen las etiquetas de la variable objetivo que tienen valores nulos
      null_values_obj_classes = X_y_[mask][self.obj_char].unique().tolist()
      for obj_tag in null_values_obj_classes : 
        # Obtención de medida de la columna filtrandolo por la variable objetivo
        measure_per_obj_tag = getattr(X_y_[X_y_[self.obj_char] == obj_tag][char], self.imputer_strategy)()

        # Reemplazo de valores nulos por la columna obtenida
        X_y_.loc[mask & (X_y_[self.obj_char] == obj_tag), char] = measure_per_obj_tag
    # for char in list(set(self.inf_chars+self.null_chars)) : 
    #   mask = X_y[char].isna()
    #   # Se obtienen las etiquetas de la variable objetivo que tienen valores nulos
    #   null_values_obj_classes = X_y[mask][self.obj_char].unique().tolist()
    #   for obj_tag in null_values_obj_classes : 
    #     # Obtención de medida de la columna filtrandolo por la variable objetivo
    #     measure_per_obj_tag = getattr(X_y[X_y[self.obj_char] == obj_tag][char], self.imputer_strategy)()

    #     # Reemplazo de valores nulos por la columna obtenida
    #     X_y.loc[mask & (X_y[self.obj_char] == obj_tag), char] = measure_per_obj_tag
    return X_y_
  
  def get_measures_for_imputer(self, X_y : pd.DataFrame) : 
    measures = {}
    for label in list(set(self.inf_chars+self.null_chars)) : 
      measures[label] = getattr(X_y[label], self.imputer_strategy)()
    self.measures = measures
    
  def chars_selection(self, X_y : pd.DataFrame) : 
    def obtener_indice_grupo(corr_list, char):
      for i, group in enumerate(corr_list):
          if char in group :
              return i
      return False  # Si no se encuentra 

    all_chars = set(self.continous_chars)

    corr_matrix = X_y[self.continous_chars].corr(method='spearman').abs()
    np.fill_diagonal(corr_matrix.values, 0)

    already_added_chars = set()
    corr_groups = []

    """ Buscar grupos de características con alta correlación """
    for char in self.continous_chars : 
      correlated_chars = list(corr_matrix.index[corr_matrix[char] > self.threshold_redundance])
      correlated_chars.append(char)
      # Eliminar duplicados
      group = list(set(correlated_chars))
      # Verificamos si la característica no está incluida entre las características ya agregadas
      matches = [char for char in group if char in already_added_chars]
      if len(group) > 1 : 
        if not matches :
          already_added_chars.update(group)
          corr_groups.append(group)
        else : 
          idx = obtener_indice_grupo(corr_groups, matches[0])
          already_added_chars.update(group)
          detected_group = set(corr_groups[idx])
          detected_group.update(group)
          corr_groups[idx] = list(detected_group) 

    """ Fusión de grupos con alta correlación con las mismas características """
    merged_groups = []

    for sublist in corr_groups : 
      sublist_set = set(sublist)
      new_groups = []
      merged_group = sublist_set.copy()
      for group in merged_groups : 
        if sublist_set & set(group) : 
          merged_group.update(group)
        else : 
          new_groups.append(merged_group)
      new_groups.append(merged_group)
      merged_groups = new_groups

    correlated_chars = set().union(*merged_groups)
    no_correlated_chars = list(all_chars-correlated_chars)

    """ Selección de características por Información Mutua """
    mi_scores = mutual_info_classif(X_y[self.categorical_chars+self.continous_chars],X_y[self.obj_char], random_state=self.seed)
    df_mi_scores = pd.DataFrame()
    df_mi_scores["char"] = self.categorical_chars+self.continous_chars
    df_mi_scores["score"] = mi_scores

    selected_chars = []
    for group in merged_groups : 
      mask = df_mi_scores['char'].isin(group)
      df_mi_scores_filtered = df_mi_scores[mask].sort_values(by='score', ascending=False)
      selected_char = df_mi_scores_filtered.iloc[0]
      selected_chars.append(selected_char['char'])

    selected_chars = selected_chars+no_correlated_chars+self.categorical_chars
    
    mask = df_mi_scores['char'].isin(selected_chars)
    final_selected_chars = df_mi_scores[mask].sort_values(by='score', ascending=False).head(10)['char'].to_list()

    categorical_selected_chars = [char for char in self.categorical_chars if char in final_selected_chars]
    continous_selected_chars   = [char for char in self.continous_chars if char in final_selected_chars]

    self.n_categorical_selected_chars = len(categorical_selected_chars)
    self.selected_chars = categorical_selected_chars+continous_selected_chars
    self.selected_indices_mi = df_mi_scores[mask].sort_values(by='score', ascending=False).head(10)
    return X_y[self.selected_chars+[self.obj_char]]

  def null_test_data_imputation(self, X_y : pd.DataFrame) : 
    X_y_ = X_y.copy()
    for k, v in self.measures.items() : 
      X_y_[k] = X_y_[k].fillna(v)
    return X_y_

  def fit(self, X_y) : 
    X_y = self.inf_data_imputation(X_y=X_y)
    X_y = self.null_train_data_imputation(X_y=X_y)
    self.get_measures_for_imputer(X_y=X_y)

    self.chars_selection(X_y=X_y)

    X_y = self.smotenc_resample(X_y=X_y)

    X_y = self.tomek_links_subsample(X_y=X_y)

    # print('Escalamiento con StandardScaler')
    # self.scaler = StandardScaler()
    # self.scaler.fit(X_y[self.selected_chars])

  def fit_transform(self, X : pd.DataFrame, y : pd.Series) : 
    X_y = X.copy()
    X_y[self.obj_char] = y
    print('Limpieza de datos')
    X_y = self.inf_data_imputation(X_y=X_y)
    X_y = self.null_train_data_imputation(X_y=X_y)
    self.get_measures_for_imputer(X_y=X_y)
    print(self.measures)

    print('Selección de características')
    X_y = self.chars_selection(X_y=X_y)
    print(self.selected_chars)

    # print('Sobremuestreo con SMOTENC')
    # X_y = self.smotenc_resample(X_y=X_y)

    # print('Submuestreo con TomekLinks')
    # X_y = self.tomek_links_subsample(X_y=X_y)

    print(X_y[self.selected_chars].shape)
    print(X_y[self.obj_char].shape)

    return X_y[self.selected_chars]
  
  def transform(self, X_y : pd.DataFrame) : 
    X_y = self.inf_data_imputation(X_y=X_y)

    X_y = self.null_test_data_imputation(X_y=X_y)
    
    return X_y[self.selected_chars]

In [4]:
class CustomResampler(BaseEstimator, SamplerMixin):
  def __init__(self, seed, obj_char='label', obj_char_majority_class='BENIGN', categorical_chars=[], labels=[]):
      # aquí defines tus parámetros si necesitas alguno
      self.seed = seed
      self.obj_char = obj_char
      self.obj_char_majority_class = obj_char_majority_class
      self.categorical_chars = categorical_chars
      self.labels = labels
        
  def smotenc_resample(self, X_y : pd.DataFrame) : 
    minority_classes = set(X_y[self.obj_char].unique()) - set([self.obj_char_majority_class])
    cant_majority_class = len(X_y[X_y[self.obj_char] == self.obj_char_majority_class])
    strategy = {
      self.obj_char_majority_class : cant_majority_class
    }
    for class_ in minority_classes : 
      calc_q = int(round(cant_majority_class/len(minority_classes), 0))
      real_q = len(X_y[X_y[self.obj_char] == class_])

      strategy[class_] = calc_q if calc_q>real_q else real_q

    if self.n_categorical_selected_chars > 0 :
      print('Se usó SMOTENC')
      smt = SMOTENC(
        categorical_features=list(range(self.n_categorical_selected_chars)),
        random_state=self.seed,
        sampling_strategy=strategy
      )
    else :
      print('Se usó SMOTE')
      smt = SMOTE(
        random_state=self.seed,
        sampling_strategy=strategy
      )

    X_resample, y_resample = smt.fit_resample(
      X_y.drop([self.obj_char], axis=1),
      X_y[self.obj_char]
    )

    df_resample = pd.DataFrame(
      X_resample,
       columns=self.selected_chars
    )

    df_resample[self.obj_char] = y_resample

    return df_resample

  def tomek_links_subsample(self, X_y : pd.DataFrame) : 
    tmk = TomekLinks(
      sampling_strategy=[self.obj_char_majority_class]
    )

    X_subsample, y_subsample = tmk.fit_resample(
      X_y.drop([self.obj_char], axis=1),
      X_y[self.obj_char]
    )
    
    df_subsample = pd.DataFrame(
      X_subsample,
       columns=self.selected_chars
    )

    df_subsample[self.obj_char] = y_subsample

    return df_subsample

  def fit_resample(self, X : pd.DataFrame, y):
    self.selected_chars = X.columns
    self.categorical_selected_chars = [char for char in self.categorical_chars if char in self.selected_chars]
    self.continous_selected_chars = list(X.drop(self.categorical_selected_chars, axis=1).columns)
    self.n_categorical_selected_chars = len(self.categorical_selected_chars) 
    self.encoder = OrdinalEncoder(categories=[self.labels])
    X = X[self.categorical_selected_chars + self.continous_selected_chars]
    X_y = pd.concat([X, y], axis=1)

    print(X_y[self.obj_char].unique())

    print('Sobremuestreo con SMOTE')
    X_y = self.smotenc_resample(X_y)
    print('SUbmuestreo con TomekLinks')
    X_y = self.tomek_links_subsample(X_y)

    y_encoded = pd.Series(
      self.encoder.fit_transform(X_y[[self.obj_char]]).ravel(),
      name=self.obj_char
    )
    return X_y[self.categorical_selected_chars + self.continous_selected_chars], y_encoded
  
  def _fit_resample(self, X, y):
    return self.fit_resample(X, y)

## 3. K-Iteraciones

### 3.1. Utils

In [5]:
# bagging_resample() : Función para muestreo Bagging
def bagging_resample(X_, y_, seed) :
  indices = np.arange(len(X_)) 
  bootstrap_indices = resample(
    indices,
    replace=True,
    stratify=y_,
    random_state=seed,
    n_samples=int(round(len(X_)*TRAIN_SIZE, 0))
  )
  # np.setdiff1d permite obtener los índices que no aparecen 
  # en los indices tras el muestreo Bootstrap
  oob_indices = np.setdiff1d(indices, bootstrap_indices)

  X_train = X_.iloc[bootstrap_indices]
  y_train = y_.iloc[bootstrap_indices]
  X_valid = X_.iloc[oob_indices]
  y_valid = y_.iloc[oob_indices]

  return X_train, y_train, X_valid, y_valid

In [6]:
def drop_duplicates(X_, y_) : 
  df = pd.DataFrame(
    columns=caracteristicas_nominales+caracteristicas_numericas,
    data=X_
  )
  df[caracteristica_objetivo] = y_
  df.drop_duplicates()
  return df[caracteristicas_nominales+caracteristicas_numericas], df[caracteristica_objetivo]

In [7]:
# save_df() : Función para guardar dataframes
def save_df(X_, y_, save_path) : 
  df_save = X_.copy()
  df_save[caracteristica_objetivo] = y_

  df_save.to_csv(save_path, index=False) 

In [8]:
df_dataset = pd.read_csv('DB/dataset.csv')

X = df_dataset.drop([caracteristica_objetivo], axis=1)
y = df_dataset[caracteristica_objetivo]

In [9]:
# Declaración de valores necesarios para K-Fold
skf = StratifiedKFold(n_splits=K_ITERS, shuffle=True, random_state=SEED)

n_models = 4
prng = np.random.RandomState(seed=SEED)
max_int32 = np.iinfo(np.int32).max
seeds_per_model = prng.randint(0, max_int32, size=n_models)
print(seeds_per_model)

# Contador de iteraciones
iter_cont = 0

[793494059 498241738 377997800 912782427]


In [10]:
bagging_model = [
  {
    'path_name' : 'rf', 
    'model_name' : 'RandomForestClassifier',
    'algorithm' : RandomForestClassifier(criterion="gini", class_weight='balanced', max_depth=20, n_estimators=100, bootstrap=True, random_state=seeds_per_model[0])
  },
  {
    'path_name' : 'dt', 
    'model_name' : 'DecisionTreeClassifier',
    'algorithm' : DecisionTreeClassifier(criterion="gini", class_weight='balanced', max_depth=20, random_state=seeds_per_model[1])
  },
  {
    'path_name' : 'mlp',
    'model_name' : 'MLPClassifier',
    'algorithm' : MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=100, early_stopping=True, random_state=seeds_per_model[2])
  },
  {
    'path_name' : 'knn',
    'model_name' : 'KNeighborsClassifier',
    'algorithm' : KNeighborsClassifier(n_neighbors=3, weights='distance', metric='manhattan')
  },
]

In [11]:
df = pd.read_csv('DB/dataset.csv')
encoder = LabelEncoder()
encoder.fit(df[caracteristica_objetivo])
LABELS = encoder.classes_ 

In [12]:
# getFrequency(): Función para obtener la distribución de frecuencias de la columna label
def getFrequency(df : pd.DataFrame, caracteristica) :
  frecuencia = df[caracteristica].value_counts()
  porcentaje = df[caracteristica].value_counts(normalize=True) * 100

  tabla_frecuencia = pd.DataFrame({
    "Frecuencia": frecuencia,
    "Frecuencia(%)": porcentaje
  })
  
  tabla_frecuencia["Frecuencia(%)"].round(2)
  
  print(tabla_frecuencia)
  print(df.shape)

In [13]:
# get_results() : Función para automatizar la generación de la Matriz de confusión y AUC de las curvas ROC y Sensibilidad Precisión
def get_results(y_test, y_pred, threshold) :
  y_pred_under_threshold = y_pred >= threshold
  y_pred_for_cm = y_pred_under_threshold.astype(int)
  """
  Matriz de confusión
              Predicted
              0     1
  Actual 0  [[TN,   FP],
         1   [FN,   TP]]
  """
  cm = confusion_matrix(y_test, y_pred_for_cm)
  auc_roc = roc_auc_score(y_test, y_pred)
  precision, recall, pr_thresholds = precision_recall_curve(y_test, y_pred)
  auc_sp = auc(recall, precision)
  
  return cm, auc_roc, auc_sp

In [14]:
# one_hot_encode() : Función para codificar la variable objetivo en razón de las clases que codificó la instancia de Machine Learning
def one_hot_encode(y, classes) : 
  one_hot = np.zeros((len(y), len(classes)))
  for i, label in enumerate(y) :
    one_hot[i, int(label)] = 1
  return one_hot

### 3.2. Ejecución

In [15]:
df_validation = pd.DataFrame(columns=["iter", "model", "label", "TP", "TN", "FP", "FN", "ms", "AUC_PS"])
df_testing    = pd.DataFrame(columns=["iter", "model", "label", "TP", "TN", "FP", "FN", "Exac", "Prec", "Sens", "F1", "ms", "AUC_PS"])

for temp_index, test_index in skf.split(X, y) : 
  X_temp, X_test = X.iloc[temp_index], X.iloc[test_index]
  y_temp, y_test = y.iloc[temp_index], y.iloc[test_index]

  X_test, y_test = drop_duplicates(X_test, y_test)

  print(f'\n    ITERACIÓN {iter_cont+1}\n')

  print('ENTRENAMIENTO')
  for n_model in range(n_models) : 
    X_train, y_train, X_valid, y_valid = bagging_resample(
      X_temp, y_temp, seeds_per_model[n_model]
    )

    print(f'MODELO {bagging_model[n_model]['model_name']}  {n_model}')
    bagging_model[n_model]['model'] = Pipeline([
        ('transformer', Transformer(
            seed=seeds_per_model[n_model], 
            categorical_chars=caracteristicas_nominales, 
            continous_chars=caracteristicas_numericas, 
            obj_char=caracteristica_objetivo, 
            obj_char_majority_class='BENIGN', 
            selected_chars=None, 
            imputer_strategy='median', 
            threshold_redundance=0.8,
            inf_chars=caracteristicas_var_max_inf, 
            null_chars=caracteristicas_var_null,
            ordered_labels=LABELS
        )),
        ('resampler', CustomResampler(
            seed=seeds_per_model[n_model], 
            obj_char=caracteristica_objetivo, 
            obj_char_majority_class='BENIGN',
            labels=LABELS
        )),
        ('scaler', StandardScaler()),
        ('algorithm', bagging_model[n_model]['algorithm'])
    ])

    print(f'Entrenamiento de la instancia {bagging_model[n_model]['model_name']}')
    bagging_model[n_model]['model'].fit(X_train, y_train)


    print(f'Validación de la instancia {bagging_model[n_model]['model_name']}')
    X_valid, y_valid = drop_duplicates(X_valid, y_valid)
    y_valid = one_hot_encode(
      encoder.transform(y_valid.values.ravel()), 
      LABELS
    )
    valid_start = time.perf_counter()
    y_valid_pred = bagging_model[n_model]['model'].predict_proba(X_valid)
    valid_end = time.perf_counter()

    for label in range(len(LABELS)) : 
      cm, auc_roc, auc_sp = get_results(
        y_test=y_valid[:,label],
        y_pred=y_valid_pred[:,label],
        threshold=0.5
      )
      df_validation.loc[len(df_validation)] = [
        f"Iteración {iter_cont+1}",
        bagging_model[n_model]["model_name"], 
        LABELS[label],
        cm[1][1], 
        cm[0][0], 
        cm[0][1], 
        cm[1][0], 
        valid_end-valid_start, 
        auc_sp
      ]
  
  print('PRUEBA')
  models_results = [
  {
    'path_name' : 'rf', 
    'model_name' : 'RandomForestClassifier',
  },
  {
    'path_name' : 'dt', 
    'model_name' : 'DecisionTreeClassifier',
  },
  {
    'path_name' : 'mlp',
    'model_name' : 'MLPClassifier',
  },
  {
    'path_name' : 'knn',
    'model_name' : 'KNeighborsClassifier',
  },
  {
    'path_name' : 'bagging',
    'model_name' : 'IDSBaggingClassifier',
  },
  ]
  y_test = one_hot_encode(
    encoder.transform(y_test.values.ravel()), 
    LABELS
  )
  bagging_start = time.perf_counter()
  y_pred_bagging = np.zeros((len(X_test), len(LABELS)))
  for i in range(n_models) : 
    model_start = time.perf_counter()
    models_results[i]['y_pred'] = bagging_model[i]['model'].predict_proba(X_test)
    model_end = time.perf_counter()
    models_results[i]['time'] = (model_end - model_start)*1000
    y_pred_bagging += models_results[i]['y_pred']
  y_pred_bagging /= len(bagging_model)
  models_results[-1]['y_pred'] = y_pred_bagging
  bagging_end = time.perf_counter()
  models_results[-1]['time'] = (bagging_end - bagging_start)*1000

  for result in models_results : 
    for i in range(len(LABELS)) :
      cm, auc_roc, auc_sp = get_results(
        y_test=y_test[:,i],
        y_pred=result["y_pred"][:,i],
        threshold=0.5
      )
      exac = (cm[1][1] + cm[0][0])/(cm[1][1]+cm[0][0]+cm[0][1]+cm[1][0])
      prec = (cm[1][1])/(cm[1][1]+cm[0][1])
      sens = (cm[1][1])/(cm[1][1]+cm[1][0])
      F1sc = (sens*prec*2)/(sens+prec)
      df_testing.loc[len(df_testing)] = [
        f"Iteración {iter_cont+1}",
        result["model_name"],
        LABELS[i],
        cm[1][1], 
        cm[0][0], 
        cm[0][1], 
        cm[1][0], 
        exac*100,
        prec*100,
        sens*100,
        F1sc*100, 
        result["time"],
        auc_sp
      ]
      """
      Matriz de confusión
                  Predicted
                  0     1
      Actual 0  [[TN,   FP],
             1   [FN,   TP]]
      """
      if LABELS[i] == 'BENIGN' : 
        exac = (cm[0][0] + cm[1][1])/(cm[0][0]+cm[1][1]+cm[1][0]+cm[0][1])
        prec = (cm[0][0])/(cm[0][0]+cm[1][0])
        sens = (cm[0][0])/(cm[0][0]+cm[0][1])
        F1sc = (sens*prec*2)/(sens+prec)
        df_testing.loc[len(df_testing)] = [
          f"Iteración {iter_cont+1}",
          result["model_name"],
          'GENERAL',
          cm[0][0], 
          cm[1][1], 
          cm[1][0], 
          cm[0][1], 
          exac*100,
          prec*100,
          sens*100,
          F1sc*100, 
          result["time"],
          auc_sp
        ]

  iter_cont += 1


    ITERACIÓN 1

ENTRENAMIENTO
MODELO RandomForestClassifier  0
Entrenamiento de la instancia RandomForestClassifier
Limpieza de datos
{'flow_bytes/s': 3903.935294499232, 'flow_packets/s': 75.45746085644218, 'flow_iat_min': 3.0, 'flow_iat_std': 17836.6592163443, 'flow_iat_max': 60257.0, 'flow_iat_mean': 17368.85}
Selección de características
['flow_duration', 'fwd_packet_length_max', 'bwd_packet_length_max', 'fwd_header_length', 'bwd_header_length', 'packet_length_max', 'packet_length_mean', 'packet_length_std', 'packet_length_variance', 'average_packet_size']
(114701, 10)
(114701,)
['BENIGN' 'DOS_HULK' 'PORTSCAN' 'DDOS' 'SSH_PATATOR' 'DOS_SLOWLORIS'
 'DOS_SLOWHTTPTEST' 'DOS_GOLDENEYE' 'FTP_PATATOR' 'BOT']
Sobremuestreo con SMOTE
Se usó SMOTE
SUbmuestreo con TomekLinks
Validación de la instancia RandomForestClassifier
MODELO DecisionTreeClassifier  1
Entrenamiento de la instancia DecisionTreeClassifier
Limpieza de datos
{'flow_bytes/s': 3917.684253765352, 'flow_packets/s': 75.45793534

In [16]:
df_validation

,iter,model,label,TP,TN,FP,FN,ms,AUC_PS
0,Iteración 1,RandomForestClassifier,BENIGN,49136,23053,18,12,0.191397,1.000000
1,Iteración 1,RandomForestClassifier,BOT,39,72180,0,0,0.191397,1.000000
2,Iteración 1,RandomForestClassifier,DDOS,5042,67175,2,0,0.191397,1.000000
3,Iteración 1,RandomForestClassifier,DOS_GOLDENEYE,409,71776,27,7,0.191397,0.994898
4,Iteración 1,RandomForestClassifier,DOS_HULK,8395,63790,4,30,0.191397,0.999876
...,...,...,...,...,...,...,...,...,...
395,Iteración 10,KNeighborsClassifier,DOS_SLOWHTTPTEST,86,72149,27,9,1.383726,0.870955
396,Iteración 10,KNeighborsClassifier,DOS_SLOWLORIS,207,72054,8,2,1.383726,0.981038
397,Iteración 10,KNeighborsClassifier,FTP_PATATOR,220,72050,1,0,1.383726,0.997738
398,Iteración 10,KNeighborsClassifier,PORTSCAN,8471,63708,84,8,1.383726,0.994732


In [17]:
df_testing

,iter,model,label,TP,TN,FP,FN,Exac,Prec,Sens,F1,ms,AUC_PS
0,Iteración 1,RandomForestClassifier,BENIGN,11569,5416,6,2,99.952922,99.948164,99.982715,99.965437,47.37626,0.999999
1,Iteración 1,RandomForestClassifier,GENERAL,5416,11569,2,6,99.952922,99.963086,99.889340,99.926199,47.37626,0.999999
2,Iteración 1,RandomForestClassifier,BOT,8,16984,0,1,99.994115,100.000000,88.888889,94.117647,47.37626,1.000000
3,Iteración 1,RandomForestClassifier,DDOS,1189,15804,0,0,100.000000,100.000000,100.000000,100.000000,47.37626,1.000000
4,Iteración 1,RandomForestClassifier,DOS_GOLDENEYE,93,16893,6,1,99.958807,93.939394,98.936170,96.373057,47.37626,0.995337
...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,Iteración 10,IDSBaggingClassifier,DOS_SLOWHTTPTEST,20,16970,0,2,99.988230,100.000000,90.909091,95.238095,419.56900,0.992087
546,Iteración 10,IDSBaggingClassifier,DOS_SLOWLORIS,50,16940,2,0,99.988230,96.153846,100.000000,98.039216,419.56900,1.000000
547,Iteración 10,IDSBaggingClassifier,FTP_PATATOR,50,16942,0,0,100.000000,100.000000,100.000000,100.000000,419.56900,1.000000
548,Iteración 10,IDSBaggingClassifier,PORTSCAN,1989,14978,24,1,99.852872,98.807750,99.949749,99.375468,419.56900,0.999569


In [18]:
df_validation.to_excel("../5_analisis_de_resultados/DB/validation_results.xlsx", index=False)
df_testing.to_excel("../5_analisis_de_resultados/DB/testing_results.xlsx", index=False)
print("Guardado")

Guardado
